## Prepare Dataset and Config Files

In [3]:
import glob


# download dataset
!wget https://github.com/abdulazeezoj/omdena-abuja-anpd/raw/main/yolov4.zip

# unzip out dataset
!unzip anpd.zip

# make train.txt and test.txt file
file_train = open('/content/anpd/train.txt', 'w')
file_test = open('/content/anpd/test.txt', 'w')

img_list = glob.glob(f"/content/anpd/obj/*.jpg", recursive=False)
dataset_size = len(img_list)
train_pct = 0.9

train_idx = round(dataset_size * train_pct)
print(f"{len(img_list)} images found")
for i, img in enumerate(img_list, 1):
    file_train.write(img.replace("\\", "/") + "\n")
    
    if i > train_idx:
        file_test.write(img.replace("\\", "/") + "\n")

file_train.close()
file_test.close()


60 images found


## Train ANPD with YOLOv4

In [ ]:
# clone darknet repository
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# build darknet
%cd darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/OPENMP=0/OPENMP=1/' Makefile

!make


**TIP**: This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit Enter
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```
Looks like this, it will click the screen every 10 minutes so that you don't get kicked off for being idle! HACKS!

*Double click to start training from last pre-trained weights*

<!-- remove the comment sign above and below
# starting training from last pre-trained weights [optional]

!./darknet detector train /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/out/anpd_best.weights -dont_show -map
change cell to code to run it-->

In [ ]:
# start training

!./darknet detector train /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/yolov4-tiny.conv.29 -dont_show -map

*Double click to resume training from last weights*

<!-- remove the comment sign above and below
# resuming training from last weights [optional]

!./darknet detector train /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/out/obj_last.weights - dont_show - map
chane cell to code to run it-->


In [ ]:
# checking mAP

!./darknet detector map /content/anpd/anpd.data /content/anpd/anpd.cfg /content/anpd/out/obj_best.weights

In [8]:
# testing the ANPD model on image
import cv2
import time


# load our input image and grab its spatial dimensions
image = cv2.imread("/content/anpd/inf/img1.jpg")

# load the objects class names our YOLO model was trained on
with open("/content/anpd/anpd.names", 'r') as f:
    classNames = f.read().splitlines()

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet("/content/anpd/anpd.cfg",
                                 "/content/anpd/out/obj_best.weights")
model = cv2.dnn_DetectionModel(net)
model.setInputParams(scale=1/255, size=(416, 416), swapRB=True)


# start detecting object and show timing information on YOLO
start = time.time()

print("[INFO] detecting object(s)...")
classIds, scores, boxes = model.detect(
    image, confThreshold=0.5, nmsThreshold=0.5)

end = time.time()
print("[INFO] YOLO took {:.6f} seconds".format(end - start))


# initialize detected bounding boxes, confidences, and class IDs, respectively
for (classId, score, box) in zip(classIds, scores, boxes):
    cv2.rectangle(image, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]),
                  color=(0, 255, 0), thickness=2)

    text = '%s: %.2f' % (classNames[classId], score)
    cv2.putText(image, text, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                color=(0, 255, 0), thickness=1)


# display result
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[INFO] loading YOLO from disk...
[INFO] detecting object(s)...
[INFO] YOLO took 0.264002 seconds


In [ ]:
# export your model's weights for future use
from google.colab import files
files.download('/content/anpd/out/obj_best.weights')
